<a href="https://colab.research.google.com/github/ChandsR/latihan_data_visualisasi/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Menyiapkan semua library yang dibutuhkan**

In [ ]:
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import pandas as pd
import plotly.express as px

# **DATA WRANGLING**

In [1]:
@st.cache_data
def load_data():
    file_path = "day.csv"  # Pastikan file berada di lokasi ini atau sesuaikan
    data = pd.read_csv(file_path)
    data['dteday'] = pd.to_datetime(data['dteday'])
    data.rename(columns={
        'yr': 'year',
        'mnth': 'month',
        'weathersit': 'weather',
        'hum': 'humidity',
        'cnt': 'total_rentals'
    }, inplace=True)
    data['is_weekend'] = data['weekday'].apply(lambda x: 1 if x in [0, 6] else 0)
    season_mapping = {1: 'Spring', 2: 'Summer', 3: 'Fall', 4: 'Winter'}
    data['season_name'] = data['season'].map(season_mapping)
    return data


NameError: name 'st' is not defined